Hello lev,
Following our talk, please see the task below.
We would love to get an answer within a week~
If you have any questions, please feel free to contact Oded/Pavel CC'd to this email.
Good luck!
Some background:
In driver monitoring system face detection is critical block in pipeline, because without proper face detection all following algorithms will fail.
Accurate face detection algorithms are slow, so to produce real time application, we use fast trackers to track found faces.
Trackers are fast but they can loose face and track another objects. To validate that tracker tracks valid face, we need fast classifier that will decide if tracker result is valid face. Valid face is face aligned face, shifted faces considered to be invalid (because other blocks in the pipeline, expect aligned faces).

Task:
You need to design and implement such system (face validator) based on Neural Networks. We prefer implementation using Keras APIs, but if you unfamiliar with Keras or have difficulties working with it, you can use tensorflow (preferred)  or any other framework for training/test neural networks.

We provide you with dataset (small part of big dataset) of valid (aligned) faces and negatives (not faces or not aligned faces).
Your implementation should be fast. Guideline real time (less then 30ms) on i7 CPU without GPU.
If you have weak CPU and it is hard to you to estimate run time on i7 CPU, lets say it should be less than 300ms on your system. It is only guideline (there is no strict limit or specific number). We will test your network with another images from the same dataset (images that were not provided to you) to test your network.

You can build your own network from the scratch or use existing classification networks or parts of them.

You should be able to describe you design process and decisions that was taken in choosing architecture and training the network. This is the most important part of the task and not the final result itself (final achieved accuracy and run time performance).
